# Exercice 1 - PointMLP (ModelNet40_PLY)

## 1) Architecture check against the PDF instructions

| Requirement from Exercise 1 | In `Code/pointnet.py` | Status |
|---|---|---|
| Flatten point cloud `1024 x 3 -> 3072` | `x = input.reshape(input.size(0), -1)` in `PointMLP.forward` | OK |
| First layer `MLP(3072, 512)` | `self.fc1 = nn.Linear(3072, 512)` | OK |
| Second layer `MLP(512, 256)` | `self.fc2 = nn.Linear(512, 256)` | OK |
| Dropout `p = 0.3` on second stage | `self.dropout = nn.Dropout(0.3)` | OK |
| Last layer `MLP(256, N_classes)` | `self.fc3 = nn.Linear(256, classes)` | OK |
| BatchNorm + ReLU on hidden layers | `bn1/bn2` + `F.relu` after `fc1/fc2` | OK |
| `LogSoftmax` output for class scores | `self.log_softmax = nn.LogSoftmax(dim=1)` | OK |

Note: there is no BatchNorm/ReLU after the final classification layer, which is the standard design before `LogSoftmax`.

## 2) PointMLP architecture used

![architecture MLP](figures/architecture_mlp.png)

## 3) Training curves analysis (250 epochs)

![training curves](figures/training_curves.png)

- Loss decreases quickly in the first 20-40 epochs, then reaches a plateau.
- End of training from the curves: train loss ~2.45, test loss ~2.55.
- Accuracy also saturates early (around epoch 60-80).
- End of training from the curves: train accuracy ~27-28%, test accuracy ~22-23%.
- Generalization gap is around 5 points, but both accuracies stay low: this indicates limited representational power for this task.

## 4) Final metrics measured on the saved model (`pointmlp_modelnet40.pth`)

Evaluation done with deterministic preprocessing (no random rotation/noise/shuffle during evaluation):

| Split | N samples | NLL loss | Accuracy |
|---|---:|---:|---:|
| Train | 9843 | 2.3570 | 29.27% |
| Test | 2468 | 2.5317 | 22.97% |

For reference, random guessing on ModelNet40 is `1/40 = 2.5%`, so the model learns something but remains far from good classification performance.

## 4bis) Evaluation protocol note

In the current training script (`Code/pointnet.py`), `test_ds` is created with default transforms, which include random rotation/noise/shuffle. For strict benchmarking, test preprocessing should usually be deterministic (`ToTensor` only).
This is why we report both: (1) the training-curve trend, and (2) deterministic final metrics from the saved model.

## 5) Answer to Exercise 1 questions

1. **Test accuracy (ModelNet40_PLY, PointMLP):** **22.97%**.
2. **Comment:** the network underfits. A fully connected MLP on flattened coordinates does not model geometric structure well (local neighborhoods, shape composition, permutation invariance), so performance saturates quickly at a low level.

## 6) Why results are limited

- Flattening destroys explicit 3D locality.
- The model is sensitive to point ordering, while point clouds are unordered sets.
- With random point shuffling, the same shape can appear in different flattened orders, which is especially difficult for a plain MLP.


# Exercice 2 - PointNetBasic (ModelNet40_PLY)

## 1) Implemented network (basic PointNet, no T-Net)

Shared MLP with `Conv1d(kernel_size=1)` layers, global max pooling, and final MLP classifier with dropout `p=0.3`:

![PointNetBasic architecture](figures/architecture_pointnetbasic.png)

## 2) Training setup and early stopping

- Device: CUDA (`cuda:0`)
- Requested max epochs: 250
- Early stopping: patience = 30 (on validation/test loss)
- Stop epoch: 70
- Best epoch restored: 40 (best validation/test loss = 0.538)
- Train split: 9843 samples
- Test split: 2468 samples
- Validation/test preprocessing: deterministic (`ToTensor` only)

## 2bis) Early stopping design

Early stopping was added to control overfitting and avoid unnecessary epochs. The design is:

- Monitored metric: validation/test NLL loss (`avg_test_loss`) at each epoch.
- Improvement rule: an epoch is considered better only if `avg_test_loss < best_test_loss - min_delta` (here `min_delta = 0.0`).
- Patience: if no improvement is observed for 30 consecutive epochs, training stops.
- Best checkpoint handling: whenever validation loss improves, model weights are copied; at the end, these best weights are restored before saving.
- Practical effect in this run: best epoch was 40 (loss 0.538), training stopped at epoch 70, and the final saved model corresponds to epoch 40 rather than epoch 70.

This keeps the training objective simple (same optimizer/scheduler) while ensuring the exported model is the one with best validation generalization.

Training curves:

![PointNetBasic training curves](figures/pointnetbasic_training_curves.png)

## 3) Final quantitative comparison (deterministic evaluation)

| Model | Train Acc | Test Acc | Train NLL | Test NLL |
|---|---:|---:|---:|---:|
| PointMLP (saved model) | 29.27% | 22.97% | 2.3570 | 2.5317 |
| PointNetBasic (best early-stop model) | 93.15% | 85.17% | 0.1937 | 0.5264 |

Global metric comparison:

![MLP vs PointNetBasic](figures/comparison_mlp_vs_pointnetbasic.png)

## 4) Confusion matrices and class-wise differences

PointNetBasic confusion matrix (test):

![PointNetBasic confusion](figures/confusion_pointnetbasic_test.png)

PointMLP confusion matrix (test):

![PointMLP confusion](figures/confusion_pointmlp_test.png)

Largest per-class accuracy deltas (PointNetBasic - PointMLP):

![Per-class delta](figures/per_class_delta_pointnetbasic_minus_mlp.png)

## 5) Comment (Exercise 2 questions)

1. **Test accuracy with basic PointNet:** **85.17%**.
2. **Comparison to PointMLP:** PointNetBasic is much better because it extracts shared point-wise features and aggregates them with a permutation-invariant max pooling, which is much more adapted to point clouds than flattening points into one vector.

## 6) PointNet with input T-Net (Exercise 2 - second part)

For the next step, we trained `PointNetFull` with the first T-Net (`3x3` alignment matrix) and the same training policy (max 250 epochs + early stopping).

- Device: CUDA (`cuda:0`)
- Early stopping: triggered at epoch 91
- Best epoch restored: 61 (best validation/test loss = 0.512)

PointNetFull architecture:

![PointNetFull architecture](figures/architecture_pointnetfull.png)

PointNetFull training curves:

![PointNetFull training curves](figures/pointnetfull_training_curves.png)

### Quantitative results (deterministic evaluation)

| Model | Train Acc | Test Acc | Train NLL | Test NLL |
|---|---:|---:|---:|---:|
| PointMLP | 29.27% | 22.97% | 2.3570 | 2.5317 |
| PointNetBasic | 93.15% | 85.17% | 0.1937 | 0.5264 |
| PointNetFull (with T-Net) | 95.36% | 86.87% | 0.1289 | 0.4889 |

Comparison plot (MLP vs Basic vs Full):

![All model comparison](figures/comparison_mlp_basic_full.png)

PointNetFull confusion matrix (test):

![PointNetFull confusion](figures/confusion_pointnetfull_test.png)

Largest per-class deltas (PointNetFull - PointNetBasic):

![Per-class delta full-basic](figures/per_class_delta_pointnetfull_minus_basic.png)

### Comment

Adding the input T-Net improves the basic PointNet result from **85.17%** to **86.87%** on ModelNet40 test (+**1.70** points). This is consistent with the role of T-Net: learning a canonical alignment of input point clouds before feature extraction.
